# Spreadsheet

    Make a spreadsheet using pinkfish.  This is useful for developing trading strategies.
    It can also be used as a tool for buy and sell signals that you then manually execute.

In [1]:
import datetime

import matplotlib.pyplot as plt
import pandas as pd

from talib.abstract import *

import pinkfish as pf
import pinkfish.itable as itable

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

# Show all columns.
pd.set_option('display.max_columns', None)

%matplotlib inline

In [2]:
# Set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)
ts.tail()

[*********************100%***********************]  1 of 1 completed


,open,high,low,close,adj_close,volume
date,,,,,,
2021-11-09,469.32,469.57,465.88,467.38,467.38,51149100
2021-11-10,465.58,467.38,462.04,463.62,463.62,69429700
2021-11-11,465.21,465.29,463.75,463.77,463.77,34848500
2021-11-12,465.12,467.86,464.11,467.27,467.27,53423300
2021-11-15,468.64,468.79,466.23,466.74,466.74,36152135


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [5]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)
ts.head()

,open,high,low,close,adj_close,volume
date,,,,,,
1993-01-29,43.97,43.97,43.75,43.94,25.72,1003200
1993-02-01,43.97,44.25,43.97,44.25,25.90,480500
1993-02-02,44.22,44.38,44.12,44.34,25.96,201300
1993-02-03,44.41,44.84,44.38,44.81,26.23,529400
1993-02-04,44.97,45.09,44.47,45.00,26.34,531500


Add technical indicators

In [6]:
# Add 200 day MA.
ts['sma200'] = pf.SMA(ts, timeperiod=200)

# 200 sma regime filter.
ts['regime'] = pf.CROSSOVER(ts, timeperiod_fast=1, timeperiod_slow=200)

# X day sma.
ts['sma70'] = pf.SMA(ts, timeperiod=70)

# Add ATR.
ts['atr'] = ATR(ts, timeperiod=14)

# Add 7 day high, and 7 day low
ts['high7'] = pd.Series(ts.close).rolling(window=7).max()
ts['low7'] = pd.Series(ts.close).rolling(window=7).min()

# Add RSI, and 2-period cumulative RSI
ts['rsi2'] = RSI(ts, timeperiod=2)
ts['c2rsi2'] = pd.Series(ts.rsi2).rolling(window=2).sum()

# Add midpoint
ts['mp'] = (ts.high + ts.low) / 2

# Add 10 day SMA of midpoint
ts['sma10'] = pd.Series(ts.mp).rolling(window=10).mean()

# Add temporary rolling 10 day Standard Deviation of midpoint
ts['__sd__'] = pd.Series(ts.mp).rolling(window=10).std()

# Add standard deviation envelope or channel around midpoint
ts['upper'] = ts.sma10 + ts['__sd__']*2
ts['lower'] = ts.sma10 - ts['__sd__']*2

# Drop temporary columns.
ts.drop(columns=['__sd__'], inplace=True)

Finalize timeseries

In [7]:
ts, start = pf.finalize_timeseries(ts, start, dropna=True)
ts.tail()

,open,high,low,close,adj_close,volume,sma200,regime,sma70,atr,high7,low7,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,,,
2021-11-09,469.32,469.57,465.88,467.38,467.38,51149100,422.93,346.00,446.49,3.81,468.93,460.04,42.57,142.22,467.73,462.53,472.85,452.21
2021-11-10,465.58,467.38,462.04,463.62,463.62,69429700,423.38,347.00,446.81,3.92,468.93,461.90,11.26,53.83,464.71,463.45,472.56,454.34
2021-11-11,465.21,465.29,463.75,463.77,463.77,34848500,423.81,348.00,447.16,3.76,468.93,463.62,16.18,27.45,464.52,464.21,472.09,456.33
2021-11-12,465.12,467.86,464.11,467.27,467.27,53423300,424.29,349.00,447.53,3.79,468.93,463.62,76.63,92.81,465.98,465.05,471.43,458.68
2021-11-15,468.64,468.79,466.23,466.74,466.74,36152135,424.74,350.00,447.87,3.70,468.93,463.62,62.87,139.49,467.51,465.86,471.00,460.71


Select a smaller time from for use with itable

In [8]:
df = ts['2021-01-01':]
df.head()

,open,high,low,close,adj_close,volume,sma200,regime,sma70,atr,high7,low7,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,,,
2021-01-04,375.31,375.45,364.82,368.79,365.22,110210800,323.50,131.00,353.26,4.55,373.88,367.57,20.37,113.05,370.14,370.28,374.34,366.23
2021-01-05,368.10,372.50,368.05,371.33,367.73,66426200,324.21,132.00,353.94,4.55,373.88,368.79,55.23,75.60,370.27,370.40,374.37,366.44
2021-01-06,369.71,376.98,369.12,373.55,369.93,107997700,324.97,133.00,354.58,4.78,373.88,368.79,74.64,129.87,373.05,370.68,374.98,366.38
2021-01-07,376.10,379.90,375.91,379.10,375.43,68766800,325.65,134.00,355.22,4.89,379.10,368.79,91.99,166.63,377.90,371.76,377.33,366.18
2021-01-08,380.59,381.49,377.10,381.26,377.57,71677200,326.32,135.00,355.92,4.86,381.26,368.79,94.78,186.77,379.29,372.84,379.64,366.05


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [9]:
pt = itable.PrettyTable(
    df, tstyle=itable.TableStyle(theme='theme1'), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(
    format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(
    format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high7'][row] == pt.df['close'][row]):
        col = df.columns.get_loc('high7')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low7'][row] == pt.df['close'][row]):
        col = df.columns.get_loc('low7')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')
    if (pt.df['regime'][row] > 0 or pt.df['close'][row] > pt.df['sma70'][row]):
        col = df.columns.get_loc('regime')
        pt.update_cell_style(rows=[row], cols=[col], color='green')

In [10]:
pt

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME,SMA200,REGIME,SMA70,ATR,HIGH7,LOW7,RSI2,C2RSI2,MP,SMA10,UPPER,LOWER
2021/01/04,375.31,375.45,364.82,368.79,365.22,110210800,323.50,131.00,353.26,4.55,373.88,367.57,20.37,113.05,370.14,370.28,374.34,366.23
2021/01/05,368.10,372.50,368.05,371.33,367.73,66426200,324.21,132.00,353.94,4.55,373.88,368.79,55.23,75.60,370.27,370.40,374.37,366.44
2021/01/06,369.71,376.98,369.12,373.55,369.93,107997700,324.97,133.00,354.58,4.78,373.88,368.79,74.64,129.87,373.05,370.68,374.98,366.38
2021/01/07,376.10,379.90,375.91,379.10,375.43,68766800,325.65,134.00,355.22,4.89,379.10,368.79,91.99,166.63,377.90,371.76,377.33,366.18
2021/01/08,380.59,381.49,377.10,381.26,377.57,71677200,326.32,135.00,355.92,4.86,381.26,368.79,94.78,186.77,379.29,372.84,379.64,366.05
2021/01/11,377.85,380.58,377.72,378.69,375.02,51034700,326.91,136.00,356.55,4.76,381.26,368.79,51.88,146.65,379.15,373.93,380.94,366.92
2021/01/12,378.89,379.86,376.36,378.77,375.10,52547700,327.53,137.00,357.14,4.67,381.26,368.79,53.19,105.07,378.11,374.56,381.85,367.27
2021/01/13,378.69,380.86,377.85,379.79,376.11,45303600,328.12,138.00,357.80,4.55,381.26,371.33,72.45,125.64,379.35,375.26,382.95,367.56
2021/01/14,380.59,381.13,378.10,378.46,374.79,49989100,328.73,139.00,358.35,4.45,381.26,373.55,34.95,107.40,379.62,375.98,383.83,368.14
2021/01/15,376.72,377.58,373.70,375.70,372.06,107160000,329.38,140.00,358.93,4.47,381.26,375.70,11.10,46.05,375.64,376.25,383.81,368.69
